Implementing uncertainty quantification with bootstrapping on a toy dataset

In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

2025-03-12 13:40:08.551848: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-12 13:40:10.160259: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-12 13:40:10.160312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-12 13:40:10.412663: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 13:40:10.837513: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# check GPU availability
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# Step 1: Generate random data
n_samples = 1000
n_features = 10

In [5]:
# Generate random input features (X) and output (y)
X = np.random.randn(n_samples, n_features)
y = 2 * np.sum(X, axis=1) + np.random.randn(n_samples)  # Simple linear relation + noise

In [6]:
X.shape, y.shape

((1000, 10), (1000,))

In [7]:
# Split the data into training, validation, and test sets (80% train, 10% validation, 10% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [8]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((800, 10), (800,), (100, 10), (100,), (100, 10), (100,))

In [15]:
# Step 2: Define a simple fully connected neural network for Phase I (bootstrap models)
def create_bootstrap_nn(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [16]:
# Step 3: Define the noise variance estimation network (Phase II)
def create_noise_variance_nn(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Output layer for variance prediction
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [11]:
# Step 4: Train bootstrap neural networks (Phase I)
def train_bootstrap_nns(X_train, y_train, B=10):
    bootstrap_models = []
    bootstrap_predictions = []

    # Generate B bootstrap samples and train models
    for _ in range(B):
        # Create a bootstrap sample by sampling with replacement
        indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_bootstrap = X_train[indices]
        y_bootstrap = y_train[indices]

        # Create and train a new model
        model = create_bootstrap_nn(X_train.shape[1:])
        model.fit(X_bootstrap, y_bootstrap, epochs=50, batch_size=32)
        bootstrap_models.append(model)

        # Store predictions on the original training data
        predictions = model.predict(X_train)
        bootstrap_predictions.append(predictions)

    return bootstrap_models, np.array(bootstrap_predictions)

In [12]:
# Step 5: Phase II - Train the noise variance estimation network (NNₑ)
def compute_r_squared(y_true, y_pred, model_variance):
    residuals = (y_true - y_pred) ** 2 - model_variance
    return np.maximum(residuals, 0)

In [13]:
def train_noise_variance_nn(X_train, y_train, bootstrap_predictions, B=10):
    # Calculate r^2(x_i) for each bootstrap model
    residuals_all = []
    for b in range(B):
        residuals = compute_r_squared(y_train, bootstrap_predictions[b], np.var(bootstrap_predictions, axis=0))
        residuals_all.append(residuals)
    
    residuals_all = np.stack(residuals_all, axis=-1)
    mean_residuals = np.mean(residuals_all, axis=-1)  # Averaging residuals across bootstrap models

    # Create and train the NNₑ for noise variance estimation
    nn_e = create_noise_variance_nn(X_train.shape[1:])
    nn_e.fit(X_train, mean_residuals, epochs=50, batch_size=32)

    return nn_e

In [17]:
# Step 6: Train the models
# Train the bootstrap models (Phase I)
bootstrap_models, bootstrap_predictions = train_bootstrap_nns(X_train, y_train, B=10)

2025-03-12 13:46:39.094175: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 495 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2025-03-12 13:46:43.669944: I external/local_xla/xla/service/service.cc:168] XLA service 0x154b91cb0b80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-03-12 13:46:43.669980: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2025-03-12 13:46:43.790382: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-03-12 13:46:44.163649: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1741805204.496006  101526 device_compiler.h:186] Compiled cluster using XLA!  This 

25/25 [==============================] - 0s 1ms/step


In [18]:
# Train the noise variance estimation model (Phase II)
nn_e = train_noise_variance_nn(X_train, y_train, bootstrap_predictions, B=10)

Epoch 1/50
25/25 [==============================] - 1s 3ms/step - loss: 18427.5469
Epoch 2/50
25/25 [==============================] - 0s 2ms/step - loss: 18094.6680
Epoch 3/50
25/25 [==============================] - 0s 2ms/step - loss: 17572.9316
Epoch 4/50
25/25 [==============================] - 0s 3ms/step - loss: 16780.4980
Epoch 5/50
25/25 [==============================] - 0s 3ms/step - loss: 15691.0771
Epoch 6/50
25/25 [==============================] - 0s 3ms/step - loss: 14418.3877
Epoch 7/50
25/25 [==============================] - 0s 3ms/step - loss: 13195.1221
Epoch 8/50
25/25 [==============================] - 0s 3ms/step - loss: 12260.7500
Epoch 9/50
25/25 [==============================] - 0s 3ms/step - loss: 11703.5596
Epoch 10/50
25/25 [==============================] - 0s 3ms/step - loss: 11420.9199
Epoch 11/50
25/25 [==============================] - 0s 3ms/step - loss: 11199.0029
Epoch 12/50
25/25 [==============================] - 0s 3ms/step - loss: 11023.9492
E

In [19]:
# Step 7: Evaluate the models on the test set
# Make predictions with the bootstrap models
bootstrap_preds_test = np.array([model.predict(X_test) for model in bootstrap_models])

4/4 [==============================] - 0s 1ms/step


In [20]:
# Calculate the mean prediction across all bootstrap models
bootstrap_mean_preds_test = np.mean(bootstrap_preds_test, axis=0)

In [21]:
# Estimate the noise variance using NNₑ
predicted_variance_test = nn_e.predict(X_test)

4/4 [==============================] - 0s 1ms/step


In [22]:
# Print results
print(f"Mean prediction for test set: {np.mean(bootstrap_mean_preds_test)}")
print(f"Estimated noise variance (NNₑ) for test set: {np.mean(predicted_variance_test)}")

Mean prediction for test set: 0.5567110180854797
Estimated noise variance (NNₑ) for test set: 76.57328796386719
